
---
<big><big><big><big><big><big>Sieci neuronowe 2018/19</big></big></big></big></big></big>

---
<big><big><big><big><big>Wsteczna propagacja</big></big></big></big></big>

---

In [8]:
# -*- coding: utf-8 -*-

import numpy as np
import pandas as pd

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter

plt.style.use("fivethirtyeight")
sns.set(font_scale=2.0)
# from bokeh.io import gridplot, output_file, show
#from bokeh.plotting import figure, output_notebook
# from bkcharts import Scatter

# Metoda spadku gradientu
2. __iteracyjna__ metoda minimalizacji kosztu $L(\theta)$
  1. start of $\theta^0$
  2. $\theta^{i+1}=\theta^i+\eta\Delta\theta^i$
  
3. __różniczkowalna__ funkcja kosztu $L()$ 
\begin{align}L(\theta^{i+1})&=L(\theta^i+\mu\Delta\theta^i)\\
&\simeq L(\theta^i)+\mu\nabla^TL(\theta^i)\Delta\theta^i
\end{align}
  * wybór $\Delta\theta^i$, że $$L(\theta^{i+1})-L(\theta^i) \simeq\eta\nabla^TL(\theta^i)\Delta\theta^i\lt0$$
  

4. kąt między gradientem a poprawką $\Delta\theta^{(i)}$ jest __rozwarty__
  * __zbyt duże__ $\eta_i\|\Delta\theta^{(i)}\|$ spowoduje, że nowe $\theta^{i+1}$ może znaleźć się po _drugiej stronie krzywizny_
  * szczególnie gdy funkcja kosztu ma bardzo różne nachylenia dla różnych parametrów (wag)
  
5. __aktualizacja__  $$\theta^{i+1}=\theta^i-\eta\nabla\,L$$
  * __liniowy__ model i __kwadratowa__ funkcja błędów
  $$\theta^{i+1}=\theta^i-\eta(y_k-\theta^{i^T}x_k)x_k$$
    * __błąd__ predykcji $(y_k-\eta^{i^T}x_k)$
    * __uczenie przez poprawianie błędów__
4. __współczynnik uczenia__ $\eta$
  * stały, mała wartość
  * __duży__ powoduje fluktuacje i oscylacje
    * problem ze zbieżnością
  * dla sieci liniowej i problemów regresji zwykle malejący w stosunku odwrotnym do kroku uczenia $t$
    * warunki konieczne dla regresji
    $$\begin{align}
    \sum_t\eta_t^2\lt\infty\\
    \sum_t\eta_t=\infty
    \end{align}$$
  * dla sieci wielowarstwowych (i innych architektur) możliwe inne procesy adaptacji
    * zwiększanie/zmniejszanie $\eta$


### Wsteczna propagacja błędów
<img src="../nn_figures/multilayer.png" width="60%"> [https://www.researchgate.net]
0. algorytm __Gradient Descent__
1. średni błąd sumą kosztów dla wszystkich przykładów, np. błąd kwadratowy
$$E(w)=\frac{1}{2N}\sum_n^NE_n=\frac{1}{2N}\sum_n^N\sum_k(y_k-\widehat{y}_k)^2$$
2. aktywacja $k$-tego neuronu to $$\widehat{y}_k=\varphi\left(\sum_{j=0}^Hw_{kj}y_j\right)$$
gdzie $y_j$ jest aktywacją j-tego neuronu w warstwie ukrytej
3. wsteczna propagacja jest __implementacją__ algorytmu spadku gradientu
  * modyfikuje wagę $w_{kj}$ (z $j$-tego do $k$-tego neuronu) __proporcjonalnie__ do pochodnej czątkowej $\partial{}E/\partial{}w_{kj}$
  $$\dfrac{\partial{}E}{\partial{}w_{kj}}=\frac{\partial{\,}E}{\partial{\,}e_k}
  \frac{\partial{\,}e_k}{\partial{\,}y_k} \frac{\partial{\,}y_k}{\partial{\,}v_k} \frac{\partial{\,}v_k}{\partial{\,}w_{kj}}$$
  gdzie $e_k=(y_k-\widehat{y}_k)$
4. poszczególne elementy
$$\begin{align}
\frac{\partial{\,}E}{\partial{\,}e_k}&=e_k\\
\frac{\partial{\,}e_k}{\partial{\,}y_k}&=-1\\
\frac{\partial{\,}y_k}{\partial{\,}v_k}&=\varphi'(v_k)\\
\frac{\partial{\,}v_k}{\partial{\,}w_{kj}}&=y_j\\
\\
\text{stąd}\\
\\
\frac{\partial{\,}E}{\partial{\,}w_{kj}}&=-e_k\varphi'(v_k)y_j
\end{align}$$
5. __reguła delta__ podaje jak modyfikować wagi
$$\Delta{}w_{kj}=-\eta\frac{\partial{\,}E}{\partial{\,}w_{kj}}=\eta\mathbf{\delta_k}y_j$$
gdzie $$\delta_k=-\frac{\partial{\,}E}{\partial{\,}v_k}=e_k\varphi'(v_k)$$
  * reguła delta odpowiada regule poprawy wag w jednowarstwowym modelu
  * $\delta_k$ odpowiada wartości błędu w jednowarstwowej sieci
6. __jak znaleźć $\delta_j$ dla neuronów ukrytych__????
  * niech $e_j$ będzie _błędem_ dla neuronu ukrytego, wtedy
  $$\begin{align}
  \delta_j&=-\frac{\partial{\,}E}{\partial{\,}e_j}\\
          &=-\frac{\partial{\,}E}{\partial{\,}y_j}\frac{\partial{\,}y_j}{\partial{\,}v_j}\\
          &=-\frac{\partial{\,}E}{\partial{\,}y_j}\varphi'(v_j)
  \end{align}$$
  * jedynym problemem jest wyliczenie $\frac{\partial{\,}E}{\partial{\,}y_j}$
  $$\begin{align}
  \frac{\partial{\,}E}{\partial{\,}y_j}
  &=\frac{1}{2}\frac{\partial}{\partial{\,}y_j}\sum_ke_k^2\\
  &=\sum_ke_k\frac{\partial{\,}e_k}{\partial{\,}y_j}\\
  &=\text{niech $t_k$ będzie prawdziwą wartością dla $x_k$}\\
  &=\sum_k\frac{\partial{\,}e_k}{\partial{\,}v_k}\frac{\partial{\,}v_k}{\partial{\,}y_j}\\
  &=\sum_ke_k\frac{\partial{\,}}{\partial{\,}v_k}(t_k-y_k)
    \frac{\partial{\,}}{\partial{\,}y_j}\left(\sum_jw_{kj}y_j\right)\\
  &=-\sum_ke_k\varphi'(v_k)\frac{\partial{\,}}{\partial{\,}y_j}\left(\sum_jw_{kj}y_j\right)\\
  &=-\sum_k\underbrace{e_k\varphi'(v_k)}_{=\delta_k}w_{kj}\\
  &=-\sum_k\delta_kw_{kj}
  \end{align}$$
  * mamy już, że 
  $$\begin{align}
  \delta_j&=-\frac{\partial{\,}E}{\partial{\,}y_j}\varphi'(v_j)\\
  &=\varphi'(v_j)\sum_k\delta_kw_{kj}
  \end{align}$$
  * stosując regułę delta dostajemy
  $$\Delta{}w_{ji}=-\eta\frac{\partial{\,}E}{\partial{\,}w_{ji}}=\eta\mathbf{\delta_j}y_i$$
  * $\delta_j$ dla neuronu ukrytego jest ważoną sumą wszystkich $\delta_k$ __kolejnej__ warstwy połączonej wagami $w_{kj}$ __wychodzącymi__ z neuronu $j$-tego
    * stąd nazwa __wstecznej propagacji błędów__

## Graf obliczeń
1. Tradycyjnie patrzy się na _warstwy_ jako grupę identycznie zbudowanych neuronów, z których każdy jest najpierw sumatorem, a potem stosuje nieliniowość
  * każdy neuron ma __własne__ parametry (wagi, biasy)
2. obecnie patrzymy jako graf operacji
  * każda jest sumatorem, nieliniowością, funkcją
  * dla każdej definiujemy 
    * __forward pass__: jak obliczana jest wartość
    * __backward pass__ sposób liczenia pochodnej
  * ogólne reguły definiują sposób obliczania gradientu funkcji kosztu po parametrach
  

### Aktywacje w modelach warstwowych
1. wsteczna propagacja sugeruje różniczkowalność
2. idealna wydaje się funkcja logistyczna
  * różniczkowalna, pochodna łatwa do obliczenia z wartości
  * jednak oddalając się od początku pochodna sbliża się do zera
  * to daje efekt długich plateau z wolnym uczeniem
  * im więcej będzie warstw, tym bardziej ten efekt będzie się skupiał na wcześniejszych (bliżej wejścia)
  * gradienty stają się też bardzo niestabilne
3. dobrym rozwiązaniem okazuje się ReLU
  * nie jest ciągła wszędzie
  * jednak gradient w części dodatniej jest stały

# Stochastyczny spadek gradientu
1. funkcja kosztu sieci neuronowej
  * wiele minimów lokalnych
  * badania pokazują, że same minima nie są największym problemem
    * w wysokich wymiarach istnieją "przejścia" między minimami
    * stochastyczność algorytmu pozwala na ucieczkę
    * dużym problemem są punkty siodłowe
3. __tryby__
  * __stochastyczny__ (_Stochastic Gradient Descent_) aktualizacja $\theta$ dla każdego przykładu
    * __szybki__ mało operacji
    * __chaotyczny__ losowe przeszukiwanie w różnych kierunkach
    * __lokalne__ poprawki
    * __GPU__ nie wykorzystane
  * __batch__ (_Gradient Descent_)
    * aktualizacja $\Delta\Theta=\frac{1}{N}\sum_i\Delta\theta(x_i)$
    * każda aktualizacja wymaga $N$ predykcji
    * __podobieństwo__ przykładów
      * niech $x_k$ i $x_j$ podobne
      * modyfikacja identyczna, stąd strata czasu
      * modyfikacja dla $x_j$ a __potem__ dla $x_k$ szybciej przybliża
    * __globalne__ poprawy
    * __GPU__ trudne dla większych zbiorów
<img src="../nn_figures/stochastic-vs-batch-gradient-descent.png" width="60%">
  * __mini-batch__ 
    * aktualizacja dla małych batchy
    * redukcja wariancji poprawek
    * __podobieństwo__ wykorzystane
    * __szybka__ zbieżność
    * __GPU__ możliwe optymalne wykorzystanie
4. Jak korzystać z SGD
  * __mieszanie__ przykładów
    * przykłady brać losowo - wystarczy mieszać po epoce
    * przykład z tej samej klasy/podobnej wartości __nie__ obok siebie
  * __preprocessing__ wstępny
    * bardzo ważne dla regularyzacji powierzchni kosztu
  * __śledzenie__ błędu trenowania i walidacji
    1. uczyć przez kilka epok
    2. błędy __trenowania__/__walidacji__ w osobnych epokach
  * __eksperymentowanie__ ze wzpółczynnikami uczącymi
    * używać małych podzbiorów przykładów
      * asymptotyczna zbieżność jest niezależna od wielkości pliku

## vanishing gradient 
  <p float="left">
  <img src="../nn_figures/training_speed_2_layers.png" width="51%" align="left">
  <img src="../nn_figures/training_speed_3_layers.png" width="51%" align="left">
  <img src="../nn_figures/training_speed_4_layers.png" width="51%" align="left">
  </p>[http://neuralnetworksanddeeplearning.com]
  
  
  * sieć o 4 neuronach kolejno po sobie (jeden neuron w jednej warstwie) z wagami
    $$(w_1, b_1)\longrightarrow(w_2, b_2)\longrightarrow(w_3, b_3)\longrightarrow(w_4, b_4)$$ 
    * $b$ to bias
    * $y_i=\sigma(v_i)=\sigma(w_iy_{i-1}+b_i)$
  * zaburzamy nieco bias $b_1$ o $\Delta b_1$. Jaki będzie wpływ na działanie sieci?
    * $f(x+\Delta x)\simeq f(x)+f'(x)\Delta x$
    * $$\begin{align}\Delta y_1&=\sigma(w_1y_0+b_1+\Delta b_1)-\sigma(w_1y_0+b_1)\\&\simeq\dfrac{\partial y1}{\partial b_1}\Delta b_1=\sigma'(v_1)\Delta b_1\end{align}$$
    * $\Delta v_2\simeq\dfrac{\partial v_2}{\partial y_1}=w_2\Delta y_1\simeq w_2\sigma'(v1)\Delta b_1$
    * $\Delta y_2\simeq\sigma'(v_2)\Delta v_2\simeq \sigma'(v_2)w_2\sigma'(v1)\Delta b_1$
    * $\Delta v_3\simeq w_3\Delta y_2\simeq w_3\sigma'(v_2)w_2\sigma'(v1)\Delta b_1$
    * $\Delta y_3\simeq\sigma'(v_3)\Delta v_3\simeq\sigma'(v_3)w_3\sigma'(v_2)w_2\sigma'(v1)\Delta b_1$
    * ...
    * $\Delta E(w)\simeq\sigma'(v_1)w_2\sigma'(v_2)w_3\sigma'(v_3)w_4\sigma'(v_4)\dfrac{\partial E}{y_4}\Delta b_1$
    * $\dfrac{\partial\Delta E(w)}{\Delta b_1}
    \simeq\sigma'(v_1)w_2\sigma'(v_2)w_3\sigma'(v_3)w_4\sigma'(v_4)\dfrac{\partial E}{y_4}$
    <img src="../nn_figures/logistic.png" width="50%">
  * jeśli zainicjalizujemy wagi na wartości $[-1, 1]$, to kazdy składnik $w_i\sigma'(v_i)<1/4$, a wiele takich składników __bardzo__ zmniejszy gradient
  * jednocześnie modyfikacja biasu gdzieś póżniej w sieci będzie odpowiadała fragmentowi ostatniego
  $$\dfrac{\partial\Delta E(w)}{\Delta b_1} \simeq\sigma'(v_1)w_2\sigma'(v_2)w_3\underbrace{\sigma'(v_3)w_4\sigma'(v_4)\dfrac{\partial E}{y_4}}_{\dfrac{\partial E}{\partial b_3}}$$
  * wagi oczywiście nie muszą być silnie mniejsze od 1
  * przy dużej liczbie neuronów wpływ wartości wag i ich zmian staje się coraz bardziej rozmyty
  * gdy będą odpowiednio wysokie, to składniki mogą stać się na tyle duże, że spowodują eksplozję gradientu

### __exploding gradient__
1. niech dana będzie sieć
  $$(w_1, b_1)\longrightarrow(w_2, b_2)\longrightarrow(w_3, b_3)\longrightarrow(w_4, b_4)$$ 
  * niech $w_1=w_2=w_3=w_4=1000$ i biasy tego nie niwelują
  * składniki $w_i\sigma'(v_i)$ w $\dfrac{\partial\Delta E(w)}{\Delta b_1}
    \simeq\sigma'(v_1)w_2\sigma'(v_2)w_3\sigma'(v_3)w_4\sigma'(v_4)\dfrac{\partial E}{y_4}$
spowodują eksplozję
  * ale jeśli $w$ wysokie, to (możliwe, że) $wx+b$ wysokie, ale wtedy $\sigma'(wx+b)$ może być bardzo małe
8. __niestabilność__
  * gradient wcześniejszych warstw jest __iloczynem__ gradientów w warstwach późniejszych
  * konieczny jest mechanizm stabilizujący
    * obcinanie gradientu
    * różne współczynniki uczenia w warstwach

## Funkcje aktywacji
<img src="../nn_figures/activation_functions.png" width="80%">
1. są niemalejące
2. wygodnie gdy są różniczkowalne - patrz backprop
3. funkcje sigmoidalne sprawiają kłopoty w sieciach głębokich
 

## Zanikający i eksplodujący gradient
1. prędkość uczenia w 
1. __vanishing gradient__
2. __exploding gradient__

## Wariacje na temat SGD
1. SGD podąża w kierunku największego spadku przy stałym $\eta$
<img src="../nn_figures/nnet-error-surface.png" width="80%">
2. nachylenie w różnych kierunkach może byc całkowicie różne
<img src="../nn_figures/lin_reg_mse.png" width="80%">[https://ml4a.github.io/ml4a/how_neural_networks_are_trained/]
